# Solution for ASTR 792, Spring 2024, HW1, problem 4

Compute the comoving volume in the two intervals 0.05<z<1 and 1<z<2 that is contained within a field whose solid angle is 0.25 deg2.  Assume $\Omega_m=0.3$, $\Omega_\Lambda=0.7$ and $h=0.7$.  I want you to write a program to solve this problem by numerically integrating E(z).  Don’t use the astropy.cosmology package nor should you use Ned Wright’s cosmology calculator.  You may want to refer to “Distance Measures in Cosmology” by Hogg for an alternate formulation of the Volume element.

I will use the notation from Hogg for $H(z) = H_0 E(z)$


In [1]:
import numpy as np
from astropy import units as u

In [58]:
#fucntion to compute DH from Hogg eq. 4
def D_H(h=0.7):
    dh = 3000. / h  #in Gpc
    return(dh)

#function to compute E(z) from Hogg eq. 14
def E_z(z, omega_m=0.3, omega_lam=0.7, omega_k=0.0):
    ez = np.sqrt(omega_m * (1+z)**3 + omega_lam)
    return(ez)  
    
#function to do the integral of 1/E(z) from Hogg eq. 15
def D_C(z, omega_m=0.3, omega_lam=0.7, omega_k=0.0, h=0.7):
    #generate an array of redshifts out to the redshift of interest
    zint = np.linspace(0.0, z, num=100)

    #this is E_z evaluated over a range of redshifts
    yint = 1./E_z(zint, omega_m=omega_m, omega_lam=omega_lam, omega_k=omega_k)
    
    #evaluate the integral for Hogg eq. 15
    int_dc = D_H(h=h) * np.trapz(yint,x=zint)

    return(int_dc)

def D_A(z, omega_m=0.3, omega_lam=0.7, omega_k=0.0, h=0.7):
    #compute the angular diameter distance at a given redshift
    #assume Omega_k = 0
    #evaluate the co-moving transverse distance for a given redshift and hubbly constant using Hogg eq. 16
    D_M = D_C(z, omega_m=omega_m, omega_lam=omega_lam, omega_k=omega_k, h=h)
    D_A = D_M / (1+z)
    
    return(D_A)
    
def dV_c(z, omega_m=0.3, omega_lam=0.7, omega_k=0.0, h=0.7):
    #compute the comoving volume element using Hogg eq. 28
    dVc = D_H(h=h) * (1+z)**2 * D_A**2 / E_z
    return(dVc)

def SA(a_sqdeg):
    #compute the solid angle in steradians from that in square degrees
    #the side of a square with the area
    degside = np.sqrt(a_sqdeg)
    #convert to rad
    rad = degside * np.pi / 180.
    #convert to steradian
    sa = rad**2
    return(sa)

def V_c(z1, z2, a_sqdeg, omega_m = 0.3, omega_lam=0.7, omega_k=0.0, h=0.7):
    '''
    calcualte the volume element between two redshifts (z1,z2) by integrating the volume element
    out to each redshift and taking the difference between the total volumes to the two redshifts.  
    Then multiply that volume, which is over the whole sky by the solid angle of the survey to get 
    the volume.
    
    a_sqdeg is the solid angle in square degrees.
    
    '''
    #the redshift arrays for the two integrals
    zint1 = np.linspace(0.0, z1, num=100)
    zint2 = np.linspace(0.0, z2, num=100)
    
    #do first for z1.  For compactness I am not going to give the cosmological parameters since I'm 
    #just using the standard cosmology
    #these return the volumes per steradian
    integrand1 = D_H(h=h) * (1+zint1)**2 * D_A(zint1)**2 / E_z(zint1)
    V1dom = np.trapz(integrand1, x=zint1)
    
    integrand2 = D_H(h=h) * (1+zint2)**2 * D_A(zint2)**2 / E_z(zint2)
    V2dom = np.trapz(integrand2, x=zint2)
    
    Vdiff = V2dom - V1dom
    
    #print('D_A = ',D_A(z1),D_A(z2))
    
    print("V2 = ",V2dom, " V1 = ", V1dom, " Vdiff = ", Vdiff)

    
    #take the difference of the volumes over the whole sky and then 
    #V = 1.0 * Vdiff  #this is in the same units as D_H
    V = SA(a_sqdeg) * Vdiff  #this is in the same units as D_H
    return(V)
    

In [77]:
z1=0.5
z2=1.0
a_sqdeg = 0.25   #the area of the survey in square degrees.
Vc = V_c(z1, z2, a_sqdeg)
print('Enclosed co-moving volume between z=%5.2f and z=%5.2f is %6.1e Mpc^3' % (z1, z2, Vc ))

z1=1.0
z2=2.0
a_sqdeg = 0.25   #the area of the survey in square degrees.
Vc = V_c(z1, z2, a_sqdeg)
print('Enclosed co-moving volume between z=%5.2f and z=%5.2f is %6.1e Mpc^3' % (z1, z2, Vc ))

V2 =  12045939938.777102  V1 =  2250254398.8190312  Vdiff =  9795685539.95807
Enclosed co-moving volume between z= 0.50 and z= 1.00 is 7.5e+05 Mpc^3
V2 =  46423648044.32488  V1 =  12045939938.777102  Vdiff =  34377708105.54778
Enclosed co-moving volume between z= 1.00 and z= 2.00 is 2.6e+06 Mpc^3


In [83]:
#check my answer using the astropy.cosmology package at https://docs.astropy.org/en/stable/api/astropy.cosmology.LambdaCDM.html#astropy.cosmology.LambdaCDM.comoving_volume
from astropy import cosmology 
fl = cosmology.LambdaCDM(70.0, 0.3, 0.7)
V1 = fl.comoving_volume(z1) / (4*np.pi)
V2 = fl.comoving_volume(z2) / (4*np.pi)
Vdiff = V2 - V1
print('Vdiff = ',Vdiff)
Vc = Vdiff * SA(a_sqdeg)
print('V in FOV = ',Vc)

Vdiff =  34306165798.575233 Mpc3
V in FOV =  2612563.927084254 Mpc3
